In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/kaggle/input/fake-news/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [4]:
df = df.dropna()

In [5]:
df.isna().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [6]:
X = df.drop('label', axis=1)
X

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [7]:
Y = df['label']
Y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [8]:
Y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [10]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM, Dense, Bidirectional

In [11]:
voc_size = 5000

In [12]:
messages = X.copy()
messages[1:5]

,id,title,author,text
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [17]:
messages.reset_index(inplace=True)

In [18]:
import nltk
import re
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

corpus = []

for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [21]:
corpus[1:3]

['flynn hillari clinton big woman campu breitbart', 'truth might get fire']

In [22]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr[1:3]

[[721, 1708, 909, 3063, 3124, 1870, 1700], [3327, 3460, 1328, 1742]]

In [25]:
sent_length = 20

embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 1471, 2716, 1314],
       [   0,    0,    0, ..., 3124, 1870, 1700],
       [   0,    0,    0, ..., 3460, 1328, 1742],
       ...,
       [   0,    0,    0, ..., 2820, 3091,  301],
       [   0,    0,    0, ...,  434,  909, 4788],
       [   0,    0,    0, ..., 1094, 1594, 1438]], dtype=int32)

In [28]:
embedding_vector_features = 40

model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [30]:
import numpy as np
X_final = np.array(embedded_docs)
Y_final = np.array(Y)

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y_final, test_size = 0.2, random_state=42)

In [34]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((14628, 20), (3657, 20), (14628,), (3657,))

In [36]:
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=10, batch_size=16)

Epoch 1/10
915/915 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9455 - loss: 0.1407 - val_accuracy: 0.9152 - val_loss: 0.2004
Epoch 2/10
915/915 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9641 - loss: 0.0913 - val_accuracy: 0.9106 - val_loss: 0.2236
Epoch 3/10
915/915 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9786 - loss: 0.0618 - val_accuracy: 0.9125 - val_loss: 0.2769
Epoch 4/10
915/915 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9883 - loss: 0.0342 - val_accuracy: 0.9147 - val_loss: 0.3339
Epoch 5/10
915/915 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9949 - loss: 0.0171 - val_accuracy: 0.9084 - val_loss: 0.4529
Epoch 6/10
915/915 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9958 - loss: 0.0117 - val_accuracy: 0.9089 - val_loss: 0.4584
Epoch 7/10
915/915 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9983 - loss: 0.0053 - val_accuracy: 0.9084 - val_loss: 0.5396
Epoch 8/10
915/915 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9994 - loss: 0.0023 - val_accuracy: 0.